In [1]:
# 使用微生物遗传算法作为搜索策略进行网络搜索test
# Standard imports
from nasbench import api
import copy
import numpy as np
import matplotlib.pyplot as plt
import random
import torch

NASBENCH_PATH = r"/workspace/Code1/nasbench/nasbench_only108.tfrecord"
nasbench = api.NASBench(NASBENCH_PATH)
# Useful constants 有用的参数
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7     # 顶点的个数
MAX_EDGES = 9  # 为了说明最多边为9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2   # Upper triangular matrix 
# 上三角矩阵 规定边
OP_SPOTS = NUM_VERTICES - 2   # Input/output vertices are fixed 输入/输出顶点是固定的
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]  # 允许的结构
ALLOWED_EDGES = [0, 1]   # Binary adjacency matrix 二元邻接矩阵

Loading dataset from file... This may take a few minutes...
Loaded dataset in 65 seconds


In [2]:
# 先 MGA 算法手动实现一下
 # 保存好的父母，让好的父母不消失，在袋子里抽两个球, 对比两个球, 把球大的放回袋子里, 把球小的变一下再放回袋子里
 # 这里直接使用的莫烦的代码主要学习他的而逻辑来俩作为搜索策略
import numpy as np
# import matplotlib.pyplot as plt

DNA_SIZE = 10            # DNA length
POP_SIZE = 20            # population size
CROSS_RATE = 0.6         # mating probability (DNA crossover)
MUTATION_RATE = 0.01     # mutation probability
N_GENERATIONS = 200
X_BOUND = [0, 5]         # x upper and lower bounds


def F(x): return np.sin(10*x)*x + np.cos(2*x)*x     # to find the maximum of this function  求这个函数的最大值


class MGA(object):
    def __init__(self, DNA_size, DNA_bound, cross_rate, mutation_rate, pop_size):
        self.DNA_size = DNA_size
        DNA_bound[1] += 1
        self.DNA_bound = DNA_bound
        self.cross_rate = cross_rate
        self.mutate_rate = mutation_rate
        self.pop_size = pop_size

        # initial DNAs for winner and loser
        self.pop = np.random.randint(*DNA_bound, size=(1, self.DNA_size)).repeat(pop_size, axis=0)

    def translateDNA(self, pop):
        # convert binary DNA to decimal and normalize it to a range(0, 5)
        return pop.dot(2 ** np.arange(self.DNA_size)[::-1]) / float(2 ** self.DNA_size - 1) * X_BOUND[1]

    def get_fitness(self, product):
        return product      # it is OK to use product value as fitness in here

    def crossover(self, loser_winner):      # crossover for loser
        cross_idx = np.empty((self.DNA_size,)).astype(np.bool)
        for i in range(self.DNA_size):
            cross_idx[i] = True if np.random.rand() < self.cross_rate else False  # crossover index
        loser_winner[0, cross_idx] = loser_winner[1, cross_idx]  # assign winners genes to loser
        return loser_winner

    def mutate(self, loser_winner):         # mutation for loser
        mutation_idx = np.empty((self.DNA_size,)).astype(np.bool)
        for i in range(self.DNA_size):
            mutation_idx[i] = True if np.random.rand() < self.mutate_rate else False  # mutation index
        # flip values in mutation points
        loser_winner[0, mutation_idx] = ~loser_winner[0, mutation_idx].astype(np.bool)
        return loser_winner

    def evolve(self, n):    # nature selection wrt pop's fitness
        for _ in range(n):  # random pick and compare n times
            sub_pop_idx = np.random.choice(np.arange(0, self.pop_size), size=2, replace=False)
            sub_pop = self.pop[sub_pop_idx]             # pick 2 from pop
            product = F(self.translateDNA(sub_pop))
            fitness = self.get_fitness(product)
            loser_winner_idx = np.argsort(fitness)
            loser_winner = sub_pop[loser_winner_idx]    # the first is loser and second is winner
            loser_winner = self.crossover(loser_winner)
            loser_winner = self.mutate(loser_winner)
            self.pop[sub_pop_idx] = loser_winner

        DNA_prod = self.translateDNA(self.pop)
        pred = F(DNA_prod)
        return DNA_prod, pred


# plt.ion()       # something about plotting
# x = np.linspace(*X_BOUND, 200)
# plt.plot(x, F(x))

ga = MGA(DNA_size=DNA_SIZE, DNA_bound=[0, 1], cross_rate=CROSS_RATE, mutation_rate=MUTATION_RATE, pop_size=POP_SIZE)

# for _ in range(N_GENERATIONS):                    # 100 generations
#     DNA_prod, pred = ga.evolve(5)          # natural selection, crossover and mutation

#     # something about plotting
#     if 'sca' in globals(): sca.remove()
#     sca = plt.scatter(DNA_prod, pred, s=200, lw=0, c='red', alpha=0.5); plt.pause(0.05)

# plt.ioff();plt.show()

In [3]:
# 正式开始使用NASbaech101实现以下MGA算法
# 创建一个随机空间
# import api
def random_spec(mean=None, var=None):
    """返回一个随机的验证空间"""
    while True:
        matrix = np.random.choice(ALLOWED_EDGES,size=(NUM_VERTICES,NUM_VERTICES))  # 第一个参数规定了矩阵的维度
        matrix = np.triu(matrix,1) # 获得矩阵的上三角 ，1对第一个对角线以下的元素归零
        ops = np.random.choice(ALLOWED_OPS,size=(NUM_VERTICES)).tolist() # 创建一个随机结构从允许结构中选择，7个顶点，并转化成列表（？）
        ops[0] = INPUT  # 将可选结构列表修改第一个改成固定结构，input
        ops[-1]  = OUTPUT # 将可选结构列表修改最后一个改成固定结构，output
        spec = api.ModelSpec(matrix=matrix, ops=ops) # 构建规定结构和初始化举证为随机矩阵的模型空间
        if nasbench.is_valid(spec):
            '''
            验证构建的模型可行性
            '''
            return spec

In [4]:
# 对于ops理解
testops = [CONV3X3, CONV1X1, MAXPOOL3X3]
test_ops = np.random.choice(testops,size=(NUM_VERTICES))# 创建一个

print(test_ops[0])

maxpool3x3


In [5]:
# 设定变异空间
# 这里变异空间也就是遗传算法中的变异规则先用官方给出示例，同样交配规则也暂时不改变，沿用官方

# 设定变异规则：变异邻接矩阵上三角，为什么是上三角，模型只存在单向连接，如果更改上三角一下的数字存在双线。
def mutate_spec(old_spec,mutation_rate=1.0):
    '''默认规定变异率为1.0'''
    while True:
        new_matrix = copy.deepcopy(old_spec.original_matrix) # 深拷贝原始矩阵
        new_ops = copy.deepcopy(old_spec.original_ops) # 同上
        # 变异规则
        # V边翻转（注意大多数最终被修减）
        edge_mutation_prob = mutation_rate / NUM_VERTICES # 边变异可能性
        for src in range(0,NUM_VERTICES - 1):
            ''''''
            for dst in range(src +1, NUM_VERTICES):
                ''''''
                if random.random() < edge_mutation_prob:
                    new_matrix[src, dst] = 1 - new_matrix[src, dst]

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from nasbench import api
import time

In [7]:
# 测试变异规则函数输出
random_spec = random_spec()
old_space = random_spec
print(old_space.original_matrix)
n_matrix = copy.deepcopy(old_space.original_matrix)
ep = 1 / 7
for src in range(0,NUM_VERTICES - 1):
    ''''''
    
    for dst in range(src +1, NUM_VERTICES):
        ''''''
        if random.random() < ep:
            n_matrix[src, dst] = 1 - n_matrix[src, dst]
print(n_matrix)

[[0 0 1 0 1 0 1]
 [0 0 1 1 1 0 0]
 [0 0 0 0 1 0 1]
 [0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[[0 0 1 1 1 0 1]
 [0 0 1 1 1 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
